In [2]:
#coding=utf-8

#1.常量使用
import tensorflow as tf


# 创建两个常量，分别为1X2,2X1
matrix1 = tf.constant([[3., 3.]])
matrix2 = tf.constant([[2.],[2.]])

# product等于两个矩阵乘积
product = tf.matmul(matrix1, matrix2)

# 启动session
sess = tf.Session()

# tf需要通过sess.run才能真实运行，并得到结果
with tf.Session() as sess:
  result = sess.run([product])
  print(result)
  print(sess.run(product))


[array([[ 12.]], dtype=float32)]
[[ 12.]]


In [5]:
#coding=utf-8

#2.tensorflow变量
import tensorflow as tf

matrix1 = tf.constant([[3., 3.]])

#申明x变量，并初始化
x = tf.Variable([[1.0, 2.0]])
new_value = tf.add(x, matrix1)
#new_value赋值给x
update = tf.assign(x, new_value)

#init_op = tf.initialize_all_variables()
#update
init_op = tf.global_variables_initializer()

sess = tf.Session()

with tf.Session() as sess:
  #需要先初始化
  sess.run(init_op)
  print(sess.run(x))
  for _ in range(3):
    #变量都需要通过sess.run执行
    sess.run(update)
    print(sess.run(x))


[[ 1.  2.]]
[[ 4.  5.]]
[[ 7.  8.]]
[[ 10.  11.]]


In [7]:
#coding=utf-8

#3.placeholder
import tensorflow as tf

#申明一个placeholder，理解为占位，shape可以填充时定，也可以事先定，run的时候填充数据
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)
output = tf.multiply(input1, input2)

with tf.Session() as sess:
  print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))
  
# output:
# [array([ 14.], dtype=float32)]


[array([ 14.], dtype=float32)]


In [9]:
#4.简单线性回归
import tensorflow as tf
import numpy as np

# Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
#生成测试数据
x_data = np.random.rand(100).astype(np.float32)
y_data = x_data * 0.1 + 0.3

# Try to find values for W and b that compute y_data = W * x_data + b
# (We know that W should be 0.1 and b 0.3, but TensorFlow will
# figure that out for us.)
#声明W、b变量，一个神经元，W随机初始化，-1.0到1.0取值
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.zeros([1]))
y = W * x_data + b

# Minimize the mean squared errors.
# loss函数定义为平方求平均
loss = tf.reduce_mean(tf.square(y - y_data))
#采用梯度下降，学习率设为0.5
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# Before starting, initialize the variables.  We will 'run' this first.
# init = tf.initialize_all_variables()
init = tf.global_variables_initializer()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in range(201):
    sess.run(train)
    if step % 20 == 0:
        #tf定义的变量都需要sess.run才可见
        print(step, sess.run(W), sess.run(b))

# Learns best fit is W: [0.1], b: [0.3]


0 [ 0.11522979] [ 0.3860535]
20 [ 0.09509154] [ 0.30249155]
40 [ 0.09893499] [ 0.30054063]
60 [ 0.09976894] [ 0.30011728]
80 [ 0.09994987] [ 0.30002546]
100 [ 0.09998912] [ 0.30000553]
120 [ 0.09999765] [ 0.3000012]
140 [ 0.09999949] [ 0.30000028]
160 [ 0.0999999] [ 0.30000007]
180 [ 0.09999995] [ 0.30000004]
200 [ 0.09999995] [ 0.30000004]


In [11]:
# 5.单层网络解决mnist数字识别
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Import data
from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('data_dir', '/tmp/data/', 'Directory for storing data')

mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

sess = tf.InteractiveSession()

# 创建模型，x是放置输入样本集，每个训练样本784个点
x = tf.placeholder(tf.float32, [None, 784])
# 定义10个神经元，
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
# 求softmax
y = tf.nn.softmax(tf.matmul(x, W) + b)

# y_样本结果集
y_ = tf.placeholder(tf.float32, [None, 10])
#loss函数定义
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Train
tf.initialize_all_variables().run()
for i in range(1000):
  # 每次取100个，随机取，迭代1000次
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x: batch_xs, y_: batch_ys})

# 比较最大的一个是否是同一个
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 求评价准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

ArgumentError: argument --data_dir: conflicting option string: --data_dir

In [12]:
# 6.多层网络解决mnist数字识别
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os.path
import time

import tensorflow as tf
from six.moves import xrange  # pylint: disable=redefined-builtin
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist


# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE


def inference(images, hidden1_units, hidden2_units):
  # 建网络模型，两层
  # 第一层网络，hidden1_units个
  with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
                            stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden1_units]),
                         name='biases')
    # 用relu作转换
    hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
  # 第二层hidden2_units个
  with tf.name_scope('hidden2'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1_units, hidden2_units],
                            stddev=1.0 / math.sqrt(float(hidden1_units))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden2_units]),
                         name='biases')
    hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
  # 再加一层
  with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden2_units, NUM_CLASSES],
                            stddev=1.0 / math.sqrt(float(hidden2_units))),
        name='weights')
    biases = tf.Variable(tf.zeros([NUM_CLASSES]),
                         name='biases')
    logits = tf.matmul(hidden2, weights) + biases
  return logits


def lossfunc(logits, labels):
  # labels是一个id列表
  labels = tf.to_int64(labels)
  # softmax转换后求cross_entropy，在求平均
  cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
      logits, labels, name='xentropy')
  loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
  return loss


def training(loss, learning_rate):
  # 利用TensorBoard输出
  tf.scalar_summary(loss.op.name, loss)
  # 用梯度下降学习.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  # 用global_step变量控制迭代次数.
  global_step = tf.Variable(0, name='global_step', trainable=False)
  train_op = optimizer.minimize(loss, global_step=global_step)
  return train_op


def evaluation(logits, labels):
  # logits，预测结果，[batch_size, NUM_CLASSES]
  # labels，样本结果，[batch_size]
  # 求top1的logits是否包含label，返回多少个正确的
  correct = tf.nn.in_top_k(logits, labels, 1)
  return tf.reduce_sum(tf.cast(correct, tf.int32))


# Basic model parameters as external flags.
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 128, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')


def placeholder_inputs(batch_size):
  # 初始化placeholder
  images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                         mnist.IMAGE_PIXELS))
  labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size))
  return images_placeholder, labels_placeholder


def fill_feed_dict(data_set, images_pl, labels_pl):
  images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size,
                                                 FLAGS.fake_data)
  feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
  }
  return feed_dict


def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
  #批量计算eval_correct
  true_count = 0  # Counts the number of correct predictions.
  steps_per_epoch = data_set.num_examples // FLAGS.batch_size
  num_examples = steps_per_epoch * FLAGS.batch_size
  for step in xrange(steps_per_epoch):
    feed_dict = fill_feed_dict(data_set,
                               images_placeholder,
                               labels_placeholder)
    true_count += sess.run(eval_correct, feed_dict=feed_dict)
  precision = true_count / num_examples
  print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))


def run_training():
  #载入数据
  data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

  # default Graph.
  with tf.Graph().as_default():
    images_placeholder, labels_placeholder = placeholder_inputs(
        FLAGS.batch_size)
    # 网络
    logits = inference(images_placeholder,
                             FLAGS.hidden1,
                             FLAGS.hidden2)
    #loss函数
    loss = lossfunc(logits, labels_placeholder)
    # 训练函数
    train_op = training(loss, FLAGS.learning_rate)
    # 评估函数
    eval_correct = evaluation(logits, labels_placeholder)

    # Build the summary Tensor based on the TF collection of Summaries.
    summary = tf.merge_all_summaries()

    init = tf.initialize_all_variables()

    # 用来保存训练checkpoint.
    saver = tf.train.Saver()

    sess = tf.Session()

    # 初始化一个SummaryWriter
    summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph)

    sess.run(init)

    for step in xrange(FLAGS.max_steps):
      start_time = time.time()
      feed_dict = fill_feed_dict(data_sets.train,
                                 images_placeholder,
                                 labels_placeholder)

      _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)

      duration = time.time() - start_time

      if step % 100 == 0:
        print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
        # Update the events file.
        summary_str = sess.run(summary, feed_dict=feed_dict)
        summary_writer.add_summary(summary_str, step)
        summary_writer.flush()

      # 保存训练结果.
      if (step + 1) % 1000 == 0 or (step + 1) == FLAGS.max_steps:
        checkpoint_file = os.path.join(FLAGS.train_dir, 'checkpoint')
        saver.save(sess, checkpoint_file, global_step=step)
        # Evaluate against the training set.
        print('Training Data Eval:')
        do_eval(sess,
                eval_correct,
                images_placeholder,
                labels_placeholder,
                data_sets.train)
        # Evaluate against the validation set.
        print('Validation Data Eval:')
        do_eval(sess,
                eval_correct,
                images_placeholder,
                labels_placeholder,
                data_sets.validation)
        # Evaluate against the test set.
        print('Test Data Eval:')
        do_eval(sess,
                eval_correct,
                images_placeholder,
                labels_placeholder,
                data_sets.test)


def main(_):
  run_training()


if __name__ == '__main__':
  tf.app.run()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data\t10k-labels-idx1-ubyte.gz


ValueError: Only call `sparse_softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)

In [ ]:
# 7.cnn解决minst数字识别
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import sys
import time

import numpy
from six.moves import urllib
from six.moves import xrange  
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

sess = tf.InteractiveSession()

x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  # 卷积过程每次移一格
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  # 2*2求max，
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# 5*5，卷积核，1个输入channel，32个输出channel
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

# 把x变为矩阵
x_image = tf.reshape(x, [-1,28,28,1])
# 卷积后作relu变换，再作一次max pool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# 5*5，卷积核，32个输入channel，64个输出channel
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])
# 打平后再映射到1024个值
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 再来一层dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# 用softmax求分类
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

# lost函数
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
# AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 准确率计算
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.initialize_all_variables())
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.8
step 300, training accuracy 0.94
step 400, training accuracy 0.92
step 500, training accuracy 0.92
step 600, training accuracy 0.88
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 0.96
step 1000, training accuracy 0.92
step 1100, training accuracy 0.9
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, tra

In [ ]:
# 8.autoencoder
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import sys
import time

import numpy as np
from six.moves import urllib
from six.moves import xrange  
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 256
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)

# 原始数据
X = tf.placeholder("float", [None, n_input])

# 两层encoder，两层decoder
weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}

# encoder过程，从784变成128个点
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# decoder,从128恢复到784个节点
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# 模型
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# loss函数，优化方法
cost = tf.reduce_mean(tf.pow(X - decoder_op, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

init = tf.initialize_all_variables()

sess = tf.InteractiveSession()
sess.run(init)

total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(training_epochs):
    # 每次跑所有的batch
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})

    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1),
              "cost=", "{:.9f}".format(c))

print("Optimization Finished!")

# 看下测试数据如何
encode_decode = sess.run(
    decoder_op, feed_dict={X: mnist.test.images[:examples_to_show]})
# 图像展示对比
f, a = plt.subplots(2, 10, figsize=(10, 2))
for i in range(examples_to_show):
    a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
    a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
f.show()
plt.draw()
plt.waitforbuttonpress()

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch: 0001 cost= 0.198353276
Epoch: 0002 cost= 0.161507845
Epoch: 0003 cost= 0.144643024
Epoch: 0004 cost= 0.136729479
Epoch: 0005 cost= 0.130499855


In [1]:
# 9.用rnn解决mnist数字识别
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import sys
import time

import numpy as np
from six.moves import urllib
from six.moves import xrange  
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 把每张图片看作一个序列，28个元素，每个元素28个点

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# 每行看成上一行的序列
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
# n_hidden个lstm单元
n_hidden = 128 
# 最终分类到0～9数字上
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# 从n_hidden个lstm到n_classes个最后一层神经元的权重
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def RNN(x, weights, biases):

    # 为rnn作数据转换
    # 输入数据是: (batch_size, n_steps, n_input)
    # 需要转换为: 'n_steps' 个矩阵 (batch_size, n_input)

    # 把n_steps和batch_size交换下，矩阵变成[n_steps,batch_size,n_input]
    x = tf.transpose(x, [1, 0, 2])
    # 再变成 (n_steps*batch_size, n_input)
    x = tf.reshape(x, [-1, n_input])
    # 拆成n_steps个 (batch_size, n_input)
    x = tf.split(0, n_steps, x)

    # n_hidden个lstm单元
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # 计算
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # 取最后一个output，rnn后再走一层网络
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# loss函数，优化方法
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 准确率评估
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # 计算准确率和loss
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter" + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # 用测试数据再评估下
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


TypeError: Input 'split_dim' of 'Split' Op has type float32 that does not match expected type of int32.

In [ ]:
# 10.word2vec


In [ ]:
url = 'http://mattmahoney.net/dc/'
def maybe_download(filename, expected_bytes): 
    #判断下下载过的就不下了
    if not os.path.exists(filename):    
        filename, _ = urllib.request.urlretrieve(url + filename, filename)  
    statinfo = os.stat(filename)  
    if statinfo.st_size == expected_bytes:    
        print('Found and verified', filename)  
    else:    
        print(statinfo.st_size)    
        raise Exception(        'Failed to verify ' + filename + '. Can you get to it with a browser?')  
    return filename

filename = maybe_download('text8.zip', 31344016)

def read_data(filename):  
    """解压缩并读取数据到数组中"""  
    with zipfile.ZipFile(filename) as f:    
        data = tf.compat.as_str(f.read(f.namelist()[0])).split() 
    return data

words = read_data(filename)
print('Data size', len(words))

In [ ]:
vocabulary_size = 50000
def build_dataset(words):  
    count = [['UNK', -1]]
    """"获取高频词"""
    count.extend(
            collections.Counter(words).most_common(
                   vocabulary_size - 1))  
    dictionary = dict()  
    """给每个高频词一个编号"""
    for word, _ in count:    
        dictionary[word] = len(dictionary)  
    data = list()  
    unk_count = 0  
    for word in words:   
        if word in dictionary:      
            index = dictionary[word]    
        else:      
            index = 0  # dictionary['UNK']      
            unk_count += 1    
    data.append(index)  #data和words对应，把词转换为下标
    count[0][1] = unk_count  #低频词个数，都算同一个字符
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))  
    return data, count, dictionary, reverse_dictionary

"""依次为所有词及下标，高频词及词频，高频词及下标，压缩词典"""
data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

In [ ]:
data_index = 0
def generate_batch(batch_size, num_skips, skip_window):   
    global data_index  
    assert batch_size % num_skips == 0  
    assert num_skips <= 2 * skip_window  
    """batch是一维，labels是二维"""
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)     
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)  
    span = 2 * skip_window + 1 # 左右各取skip_window个词 
    buffer = collections.deque(maxlen=span)  
    for _ in range(span):    # 依次取span个词
        buffer.append(data[data_index])    
        data_index = (data_index + 1) % len(data)  
    for i in range(batch_size // num_skips):    
        target = skip_window  # 目标词是中间那个
        targets_to_avoid = [ skip_window ]    
        for j in range(num_skips):      ＃从目标次左右取num_skips个
            while target in targets_to_avoid:        
                target = random.randint(0, span - 1)  
            targets_to_avoid.append(target)      
            batch[i * num_skips + j] = buffer[skip_window] 
            labels[i * num_skips + j, 0] = buffer[target]   
        buffer.append(data[data_index])   #deque挤掉最前面的 
        data_index = (data_index + 1) % len(data)  
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):  
    print(batch[i], reverse_dictionary[batch[i]],      '->', labels[i, 0], reverse_dictionary[labels[i, 0]])


In [ ]:
batch_size = 128
embedding_size = 128  # 词向量维度.
skip_window = 1       # 左右窗口大小.
num_skips = 2         #每个窗口取几个词
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():  
    """placeholder用来放置网络使用过程的数据"""
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])  
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])  
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32) 

    with tf.device('/cpu:0'):  
        """词向量，二维，词典大小＊词向量维数"""
        embeddings = tf.Variable(        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0)) 
        """根据train_inputs查找embedding"""   
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)    
       """构造网络"""
       nce_weights = tf.Variable(        tf.truncated_normal([vocabulary_size, embedding_size],                            stddev=1.0 / math.sqrt(embedding_size)))    
       nce_biases = tf.Variable(tf.zeros([vocabulary_size]))  
       """定义lost function，"""
       loss = tf.reduce_mean(      
                tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,                     num_sampled, vocabulary_size))  
       """定义优化方法"""     
       optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)  
      """norm化，每一行平方求和再开方.  """
      norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))  
      normalized_embeddings = embeddings / norm   
      """找到评估的几个词向量"""
      valid_embeddings = tf.nn.embedding_lookup(      normalized_embeddings, valid_dataset)  
      """相似度矩阵，得到每个待评估的词和所有词的相似度"""
      similarity = tf.matmul(      valid_embeddings, normalized_embeddings, transpose_b=True)  
      # Add variable initializer.  
      init = tf.initialize_all_variables()

In [ ]:
num_steps = 100001
with tf.Session(graph=graph) as session:  
     """初始化所有变量"""
    init.run()  
    print("Initialized")  
    average_loss = 0  
    for step in xrange(num_steps):    
        batch_inputs, batch_labels = generate_batch(        batch_size, num_skips, skip_window)    
        feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}    
        """运行依次迭代，指定loss函数，训练方法，初始数据"""  
        _,loss_val = session.run([optimizer, loss], feed_dict=feed_dict)    
        average_loss += loss_val    
        if step % 2000 == 0:      
            if step > 0:        
                average_loss /= 2000      
        # The average loss is an estimate of the loss over the last 2000 batches.      
            print("Average loss at step ", step, ": ", average_loss)  
            average_loss = 0    
        # Note that this is expensive (~20% slowdown if computed every 500 steps)    
        if step % 10000 == 0:      
            """计算similarity，结果是[评估个数＊词数]"""
            sim = similarity.eval()      
            for i in xrange(valid_size):        
                valid_word = reverse_dictionary[valid_examples[i]]      
                top_k = 8 # number of nearest neighbors  
                """每个词的top_k个最相似词"""      
                nearest = (-sim[i, :]).argsort()[1:top_k+1]        
                log_str = "Nearest to %s:" % valid_word        
                for k in xrange(top_k):          
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)  
    final_embeddings = normalized_embeddings.eval()